# 写真を表す

In [4]:
import json
import os
from PIL import Image, ImageDraw

# Expand user home directory and construct the paths
json_file_path = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\json_results\page_0_result.json'  # Update this path
images_dir = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\raw\01.jpg'  # Update this path

test_save_path = "./"

# Load the JSON data
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Extract the text coordinates
text_coords = data["texts"]

# Open the image
with Image.open(images_dir) as img:
    draw = ImageDraw.Draw(img)

    # Draw red boxes for each set of coordinates
    for box in text_coords:
        # box format: [x1, y1, x2, y2]
        draw.rectangle(box, outline="red", width=5)

    # Save the modified image with a new name
    modified_image_path = os.path.join(test_save_path, f'modified_{os.path.basename(images_dir)}')
    img.save(modified_image_path)

print(f"Saved modified image: {modified_image_path}")

Saved modified image: ./modified_01.jpg


# Create box to hide bubble chat

In [7]:
import json
import os
import time
from PIL import Image, ImageDraw

# Expand user home directory and construct the paths
json_file_path = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\json_results\page_0_result.json'  # Update this path
images_dir = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\raw\01.jpg'  # Update this path
test_save_path = "./"

# Load the JSON data
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Extract the text coordinates
text_coords = data["texts"]

# Open the image
with Image.open(images_dir) as img:
    draw = ImageDraw.Draw(img)

    # Create a modified copy of the image for erasing text
    modified_image = img.copy()

    # Loop through each set of coordinates and erase text
    for box in text_coords:
        # Get corner coordinates
        top_left = (box[0], box[1])
        top_right = (box[2], box[1])
        bottom_left = (box[0], box[3])
        bottom_right = (box[2], box[3])

        # Get the colors at the corners
        colors = [
            img.getpixel(top_left),
            img.getpixel(top_right),
            img.getpixel(bottom_left),
            img.getpixel(bottom_right),
        ]

        # Find the color with the maximum value (highest sum of RGB values)
        max_color = max(colors, key=lambda c: sum(c[:3]))  # Ignore alpha if present

        # Fill the rectangle with the center color
        draw.rectangle(box, fill=max_color)

    # Save the final modified image
    modified_image_path = os.path.join(test_save_path, f'modified_{os.path.basename(images_dir)}')
    img.save(modified_image_path)

print(f"Saved modified image: {modified_image_path}")


Saved modified image: ./modified_01.jpg


In [14]:
import json
import os
from PIL import Image, ImageDraw

# Expand user home directory and construct the paths
json_file_path = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\json_results\page_0_result.json'  # Update this path
images_dir = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\raw\01.jpg'  # Update this path
test_save_path = "./full_image"
os.makedirs(test_save_path, exist_ok=True)

# Load the JSON data
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Extract the text coordinates
text_coords = data["texts"]

# Open the image
with Image.open(images_dir) as img:
    # Create a copy of the image for drawing
    original_img = img.copy()
    img.save(os.path.join(test_save_path, f'0_modified_{os.path.basename(images_dir)}'))
    
    # Loop through each set of coordinates in reverse order
    for i, box in enumerate(reversed(text_coords)):
        # box format: [x1, y1, x2, y2]

        # Get corner coordinates
        top_left = (box[0], box[1])
        top_right = (box[2], box[1])
        bottom_left = (box[0], box[3])
        bottom_right = (box[2], box[3])

        # Get the colors at the corners
        colors = [
            original_img.getpixel(top_left),
            original_img.getpixel(top_right),
            original_img.getpixel(bottom_left),
            original_img.getpixel(bottom_right),
        ]

        # Find the color with the maximum value (highest sum of RGB values)
        max_color = max(colors, key=lambda c: sum(c[:3]))  # Ignore alpha if present

        # Create a draw object on the original image copy
        draw = ImageDraw.Draw(img)

        # Fill the rectangle with the max color
        draw.rectangle(box, fill=max_color)

        # Save the modified image with a unique name
        modified_image_path = os.path.join(test_save_path, f'{i+1}_modified_{os.path.basename(images_dir)}')
        img.save(modified_image_path)

        # Optionally, you might want to keep the original image for the next iteration
        # img = original_img.copy()  # Reset the image for the next box

print(f"Saved modified images for each speech bubble.")


Saved modified images for each speech bubble.


# Works with panel

In [5]:
import json
import os
from PIL import Image, ImageDraw

# Expand user home directory and construct the paths
json_file_path = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\json_results\page_0_result.json'  # Update this path
images_dir = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\raw\01.jpg'  # Update this path
test_save_path = "./panel_images"
os.makedirs(test_save_path, exist_ok=True)

# Load the JSON data
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Extract the text coordinates and panel coordinates
text_coords = data["texts"]
panel_coords = data["panels"]
total_length_text = len(text_coords)
image_name_ext = os.path.basename(images_dir)

# Split the image name and its extension
img_name, img_extension = os.path.splitext(image_name_ext)

with Image.open(images_dir) as img:
    # Create a copy of the image for drawing
    original_img = img.copy()

    # Loop through each panel
    for panel_index, panel in enumerate(panel_coords):
        # Create a panel-specific image by cropping the original image
        panel_box = (panel[0], panel[1], panel[2], panel[3])  # box format: (x1, y1, x2, y2)
        panel_image = original_img.crop(panel_box)
        original_panel_image = original_img.crop(panel_box)

        # Create a draw object on the panel image copy
        draw = ImageDraw.Draw(panel_image)

        # Initialize the modified panel image path
        modified_panel_image_path = os.path.join(test_save_path, f'page_{img_name}_panel_{panel_index}_bubble_{{}}.{img_extension}')

        # Loop through each set of coordinates in reverse order
        for box_index, box in enumerate(reversed(text_coords)):
            # box format: [x1, y1, x2, y2]
            # Calculate the center point of the chatbox
            x_center = (box[0] + box[2]) // 2
            y_center = (box[1] + box[3]) // 2

            # Check if the center point is inside the panel
            if panel[0] <= x_center <= panel[2] and panel[1] <= y_center <= panel[3]:
                # Calculate the position of the box relative to the panel
                relative_box = (
                    box[0] - panel[0],
                    box[1] - panel[1],
                    box[2] - panel[0],
                    box[3] - panel[1]
                )

                # Get corner coordinates
                top_left = (relative_box[0], relative_box[1])
                top_right = (relative_box[2], relative_box[1])
                bottom_left = (relative_box[0], relative_box[3])
                bottom_right = (relative_box[2], relative_box[3])

                # Get the colors at the corners
                colors = [
                    original_img.getpixel(top_left),
                    original_img.getpixel(top_right),
                    original_img.getpixel(bottom_left),
                    original_img.getpixel(bottom_right),
                ]

                # Find the color with the maximum value (highest sum of RGB values)
                max_color = max(colors, key=lambda c: sum(c[:3]))  # Ignore alpha if present

                # Fill the rectangle with the max color
                draw.rectangle(relative_box, fill=max_color)

                # Save modified image path for the current chat bubble
                modified_image_path = modified_panel_image_path.format(total_length_text - (box_index + 1))

                # Instead of saving here, save it once at the end
                panel_image.save(modified_image_path)

        # Save the modified panel image with the last computed bubble index
        original_panel_image.save(modified_panel_image_path.format(total_length_text))

print(f"Saved modified panel images with speech bubbles.")

Saved modified panel images with speech bubbles.


Function

In [18]:
import json
import os
from PIL import Image, ImageDraw

def process_image_and_json(images_dir: str, json_file_path: str, test_save_path: str = "./panel_images", name_format: str = "page_{}_panel_{}_bubble_{}.{}"):
    os.makedirs(test_save_path, exist_ok=True)

    # Load the JSON data
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # Extract the text coordinates and panel coordinates
    text_coords = data["texts"]
    panel_coords = data["panels"]
    total_length_text = len(text_coords)
    image_name_ext = os.path.basename(images_dir)

    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)

    with Image.open(images_dir) as img:
        # Create a copy of the image for drawing
        original_img = img.copy()

        # Loop through each panel
        for panel_index, panel in enumerate(panel_coords):
            # Create a panel-specific image by cropping the original image
            panel_box = (panel[0], panel[1], panel[2], panel[3])  # box format: (x1, y1, x2, y2)
            panel_image = original_img.crop(panel_box)

            # Create a draw object on the panel image copy
            draw = ImageDraw.Draw(panel_image)

            # Initialize the modified panel image path
            modified_panel_image_path = os.path.join(test_save_path, name_format)
            text_order = total_length_text

            # Save the modified panel image with the last computed bubble index
            panel_image.save(modified_panel_image_path.format(image_name, panel_index, text_order, image_extension))
            
            # Loop through each set of coordinates in reverse order
            for box_index, box in enumerate(reversed(text_coords)):
                # box format: [x1, y1, x2, y2]
                # Calculate the center point of the chatbox
                x_center = (box[0] + box[2]) // 2
                y_center = (box[1] + box[3]) // 2

                # Check if the center point is inside the panel
                if panel[0] <= x_center <= panel[2] and panel[1] <= y_center <= panel[3]:
                    # Calculate the position of the box relative to the panel
                    relative_box = (
                        box[0] - panel[0],
                        box[1] - panel[1],
                        box[2] - panel[0],
                        box[3] - panel[1]
                    )

                    # Get corner coordinates
                    top_left = (relative_box[0], relative_box[1])
                    top_right = (relative_box[2], relative_box[1])
                    bottom_left = (relative_box[0], relative_box[3])
                    bottom_right = (relative_box[2], relative_box[3])

                    # Get the colors at the corners
                    colors = [
                        original_img.getpixel(top_left),
                        original_img.getpixel(top_right),
                        original_img.getpixel(bottom_left),
                        original_img.getpixel(bottom_right),
                    ]

                    # Find the color with the maximum value (highest sum of RGB values)
                    max_color = max(colors, key=lambda c: sum(c[:3]))  # Ignore alpha if present

                    # Fill the rectangle with the max color
                    draw.rectangle(relative_box, fill=max_color)

                    # Save modified image path for the current chat bubble
                    text_order = total_length_text - (box_index + 1)
                    modified_image_path = modified_panel_image_path.format(image_name, panel_index, text_order, image_extension)

                    # Instead of saving here, save it once at the end
                    panel_image.save(modified_image_path)


    print(f"Processed and saved modified panel images for: {image_name_ext}")

# Example usage
json_file_path = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\json_results\page_1_result.json'  # Update this path
images_dir = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\raw\01.jpg'  # Update this path
test_save_path = "./panel_images"
name_format = "page_{}_panel_{}_bubble_{}.{}"
process_image_and_json(images_dir = images_dir, json_file_path = json_file_path, test_save_path = test_save_path, name_format = name_format)

Processed and saved modified panel images for: 01.jpg


Work with folder

In [24]:
import json
import os
from PIL import Image, ImageDraw

def process_image_and_json(images_dir: str, json_file_path: str, save_path: str = "./panel_images", name_format: str = "page_{}_panel_{}_bubble_{}.{}"):
    os.makedirs(save_path, exist_ok=True)

    # Load the JSON data
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # Extract the text coordinates and panel coordinates
    text_coords = data["texts"]
    panel_coords = data["panels"]
    total_length_text = len(text_coords)
    image_name_ext = os.path.basename(images_dir)
    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)

    with Image.open(images_dir) as img:
        # Create a copy of the image for drawing
        original_img = img.copy()

        # Loop through each panel
        for panel_index, panel in enumerate(panel_coords):
            # Create a panel-specific image by cropping the original image
            panel_box = (panel[0], panel[1], panel[2], panel[3])  # box format: (x1, y1, x2, y2)
            panel_image = original_img.crop(panel_box)

            # Create a draw object on the panel image copy
            draw = ImageDraw.Draw(panel_image)

            # Initialize the modified panel image path
            modified_panel_image_path = os.path.join(save_path, name_format)
            text_order = total_length_text

            # Save the modified panel image with the last computed bubble index
            panel_image.save(modified_panel_image_path.format(image_name, panel_index, text_order, image_extension))
            
            # Loop through each set of coordinates in reverse order
            for box_index, box in enumerate(reversed(text_coords)):
                # box format: [x1, y1, x2, y2]
                # Calculate the center point of the chatbox
                x_center = (box[0] + box[2]) // 2
                y_center = (box[1] + box[3]) // 2

                # Check if the center point is inside the panel
                if panel[0] <= x_center <= panel[2] and panel[1] <= y_center <= panel[3]:
                    # Calculate the position of the box relative to the panel
                    relative_box = (
                        box[0] - panel[0],
                        box[1] - panel[1],
                        box[2] - panel[0],
                        box[3] - panel[1]
                    )

                    # Get corner coordinates
                    top_left = (relative_box[0], relative_box[1])
                    top_right = (relative_box[2], relative_box[1])
                    bottom_left = (relative_box[0], relative_box[3])
                    bottom_right = (relative_box[2], relative_box[3])

                    # Get the colors at the corners
                    colors = [
                        original_img.getpixel(top_left),
                        original_img.getpixel(top_right),
                        original_img.getpixel(bottom_left),
                        original_img.getpixel(bottom_right),
                    ]

                    # Find the color with the maximum value (highest sum of RGB values)
                    max_color = max(colors, key=lambda c: sum(c[:3]))  # Ignore alpha if present

                    # Fill the rectangle with the max color
                    draw.rectangle(relative_box, fill=max_color)

                    # Save modified image path for the current chat bubble
                    text_order = total_length_text - (box_index + 1)
                    modified_image_path = modified_panel_image_path.format(image_name, panel_index, text_order, image_extension)

                    # Instead of saving here, save it once at the end
                    panel_image.save(modified_image_path)

    print(f"Processed and saved modified panel images for: {image_name_ext}")

def process_all_images_and_jsons(images_folder: str, json_folder: str, save_path: str = "./panel_images", name_format: str = "page_{}_panel_{}_bubble_{}.{}"):
    # Get all image and json file names
    image_files = [f for f in os.listdir(images_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    json_files = [f for f in os.listdir(json_folder) if f.lower().endswith('.json')]
    if len(image_files) != len(json_files):
        print("Number of images and json files do not match!")
    else:
        for image_file in image_files:
            # Construct full image path
            image_path = os.path.join(images_folder, image_file)

            # Corresponding json file name (assuming they have the same base name)
            json_file_name = os.path.splitext(image_file)[0] + '.json'
            if json_file_name in json_files:
                json_path = os.path.join(json_folder, json_file_name)

                # Process the image and json
                process_image_and_json(image_path, json_path, save_path, name_format)

# Example usage
images_folder = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\raw'  # Update this path
json_folder = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\json_results'  # Update this path
save_path = "./panel_images_full_chapter"
name_format = "page_{}_panel_{}_bubble_{}.{}"

process_all_images_and_jsons(images_folder, json_folder, save_path, name_format)


images: ['00.jpg', '01.jpg', '02.jpg', '03.jpg', '04.jpg', '05.jpg', '06.jpg', '07.jpg', '08.jpg', '09.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg']
jsons: ['00.json', '01.json', '02.json', '03.json', '04.json', '05.json', '06.json', '07.json', '08.json', '09.json', '10.json', '11.json', '12.json', '13.json', '14.json', '15.json', '16.json', '17.json', '18.json', '19.json', '20.json', '21.json', '22.json', '23.json', '24.json', '25.json', '26.json', '27.json', '28.json', '29.json', '30.json', '31.json', '32.json', '33.json', '34.json', '35.json', '36.json', '37.json', '38.json', '39.json', '40.json', '41.json', '42.json', '43.json', '44.json', '45.json']
Processed and sav

# Images to Video

In [41]:
import cv2
import os
from glob import glob

# Define the directory where the modified images are stored
image_dir = './panel_images'  # Change this to your actual save path
output_video_path = './panel_images/ruri_dragon.mp4'  # Change the extension to .mp4
fps = 30  # Desired frames per second for the video
duration_per_image = 0.5  # Duration each image is displayed in seconds

# Calculate the total frames for each image based on desired display duration
frames_per_image = int(fps * duration_per_image)

# Get all the modified images sorted in reverse order
image_files = sorted(glob(os.path.join(image_dir, 'page_*_panel_*_bubble_*.jpg')), reverse=False)

# Check if there are any image files
if not image_files:
    print("No modified images found.")
else:
    # Get the dimensions of the first image to set the video size
    first_image = cv2.imread(image_files[0])
    height, width, _ = first_image.shape

    # Create a VideoWriter object for MP4
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Loop through the images and write them to the video
    for image_file in image_files:
        img = cv2.imread(image_file)
        for _ in range(frames_per_image):
            video_writer.write(img)  # Write the same image multiple times for the duration

    # Release the video writer
    video_writer.release()

    print(f"Video saved as: {output_video_path}")

Video saved as: ./panel_images/ruri_dragon.mp4


The above code only works with images with same height/width. To fix it there is 2 approaches:

Option 1: Fixed Video Size by zooming smaller images to fit either height or width

In [6]:
import cv2
import os
import numpy as np
from glob import glob

# Define the directory where the modified images are stored
image_dir = './panel_images'  # Change this to your actual save path
output_video_path = './panel_images/ruri_dragon.mp4'  # Change the extension to .mp4
fps = 30  # Desired frames per second for the video
duration_per_image = 0.5  # Duration each image is displayed in seconds

# Calculate the total frames for each image based on desired display duration
frames_per_image = int(fps * duration_per_image)

# Get all the modified images sorted in reverse order
image_files = sorted(glob(os.path.join(image_dir, 'page_*_panel_*_bubble_*.jpg')), reverse=False)

# Check if there are any image files
if not image_files:
    print("No modified images found.")
else:
    # Find the largest dimensions among the images
    max_height = 0
    max_width = 0

    for image_file in image_files:
        img = cv2.imread(image_file)
        height, width, _ = img.shape
        max_height = max(max_height, height)
        max_width = max(max_width, width)

    # Create a VideoWriter object for MP4 with the maximum dimensions
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (max_width, max_height))

    # Loop through the images and write them to the video
    for image_file in image_files:
        img = cv2.imread(image_file)

        # Get the dimensions of the current image
        height, width, _ = img.shape
        
        # Calculate the scaling factor to fit the image to the maximum width or height
        scale = min(max_width / width, max_height / height)
        new_width = int(width * scale)
        new_height = int(height * scale)

        # Resize the image
        resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

        # Create a black background image with max dimensions
        background = np.zeros((max_height, max_width, 3), dtype=np.uint8)
        
        # Calculate the position to place the resized image on the background
        x_offset = (max_width - new_width) // 2
        y_offset = (max_height - new_height) // 2
        
        # Place the resized image on the background
        background[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_img

        for _ in range(frames_per_image):
            video_writer.write(background)  # Write the padded image multiple times for the duration

    # Release the video writer
    video_writer.release()

    print(f"Video saved as: {output_video_path}")


Video saved as: ./panel_images/ruri_dragon.mp4


Using Padding

In [7]:
import cv2
import os
from glob import glob

# Define the directory where the modified images are stored
image_dir = './panel_images'  # Change this to your actual save path
output_video_path = './panel_images/ruri_dragon_padding.mp4'  # Change the extension to .mp4
fps = 30  # Desired frames per second for the video
duration_per_image = 0.5  # Duration each image is displayed in seconds

# Calculate the total frames for each image based on desired display duration
frames_per_image = int(fps * duration_per_image)

# Get all the modified images sorted in reverse order
image_files = sorted(glob(os.path.join(image_dir, 'page_*_panel_*_bubble_*.jpg')), reverse=False)

# Check if there are any image files
if not image_files:
    print("No modified images found.")
else:
    # Find the largest dimensions among the images
    max_height = 0
    max_width = 0

    for image_file in image_files:
        img = cv2.imread(image_file)
        height, width, _ = img.shape
        max_height = max(max_height, height)
        max_width = max(max_width, width)

    # Create a VideoWriter object for MP4 with the maximum dimensions
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (max_width, max_height))

    # Loop through the images and write them to the video
    for image_file in image_files:
        img = cv2.imread(image_file)
        
        # Create a black background image with max dimensions
        background = np.zeros((max_height, max_width, 3), dtype=np.uint8)
        
        # Get the dimensions of the current image
        height, width, _ = img.shape
        
        # Calculate the position to place the current image on the background
        x_offset = (max_width - width) // 2
        y_offset = (max_height - height) // 2
        
        # Place the current image on the background
        background[y_offset:y_offset + height, x_offset:x_offset + width] = img

        for _ in range(frames_per_image):
            video_writer.write(background)  # Write the padded image multiple times for the duration

    # Release the video writer
    video_writer.release()

    print(f"Video saved as: {output_video_path}")

Video saved as: ./panel_images/ruri_dragon_padding.mp4


Function

In [26]:
import cv2
import os
import numpy as np
from glob import glob

def create_video_from_images(image_dir: str, output_video_path: str, name_format: str, use_padding: bool, fps: int = 30, duration_per_image: float = 0.5, reverse_order: bool = False):
    output_video_name = os.path.join(output_video_path, f'video_Padding_{use_padding}.mp4')
    # Calculate the total frames for each image based on desired display duration
    frames_per_image = int(fps * duration_per_image)

    # Get all the modified images sorted in reverse order using the provided name_format
    image_files = sorted(glob(os.path.join(image_dir, name_format.format('*', '*', '*', '*'))), reverse=reverse_order)

    # Check if there are any image files
    if not image_files:
        print("No modified images found.")
        return

    # Find the largest dimensions among the images
    max_height = 0
    max_width = 0

    for image_file in image_files:
        img = cv2.imread(image_file)
        height, width, _ = img.shape
        max_height = max(max_height, height)
        max_width = max(max_width, width)

    # Create a VideoWriter object for MP4 with the maximum dimensions
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    video_writer = cv2.VideoWriter(output_video_name, fourcc, fps, (max_width, max_height))

    # Loop through the images and write them to the video
    for image_file in image_files:
        img = cv2.imread(image_file)

        # Create a black background image with max dimensions if padding is needed
        if use_padding:
            background = np.zeros((max_height, max_width, 3), dtype=np.uint8)

            # Get the dimensions of the current image
            height, width, _ = img.shape
            
            # Calculate the position to place the current image on the background
            x_offset = (max_width - width) // 2
            y_offset = (max_height - height) // 2
            
            # Place the current image on the background
            background[y_offset:y_offset + height, x_offset:x_offset + width] = img
        else:
            # For no padding, fit the image to max dimensions
            height, width, _ = img.shape
            
            # Calculate the scaling factor to fit the image to the maximum width or height
            scale = min(max_width / width, max_height / height)
            new_width = int(width * scale)
            new_height = int(height * scale)

            # Resize the image
            resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

            # Create a black background image with max dimensions
            background = np.zeros((max_height, max_width, 3), dtype=np.uint8)
            
            # Calculate the position to place the resized image on the background
            x_offset = (max_width - new_width) // 2
            y_offset = (max_height - new_height) // 2
            
            # Place the resized image on the background
            background[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_img

        for _ in range(frames_per_image):
            video_writer.write(background)  # Write the (padded) image multiple times for the duration

    # Release the video writer
    video_writer.release()

    print(f"Video saved as: {output_video_name}")

# Example usage
image_dir = "./panel_images_full_chapter"
save_path = "./panel_images_full_chapter"
name_format = "page_{}_panel_{}_bubble_{}.{}"

# Create video without padding
# create_video_from_images(image_dir, output_video_path_no_padding, name_format, use_padding=False)

# Create video with padding
create_video_from_images(image_dir, save_path, name_format, use_padding=True)

Video saved as: ./panel_images_full_chapter\video_True.mp4


Option 2: Create multiple videos and then joins them later

# Show custom box on images to check

In [31]:
import json
import os
from PIL import Image, ImageDraw

images_dir = r'D:\Download\Tu_Lieu\Cao_Hoc\Ky_3\Multimedia\FinalExam\magi_functional\data_test\personal_data\Ruri_Dragon\raw\01.jpg'  # Update this path

test_save_path = "./"


# Extract the text coordinates
text_coords = [[613.284423828125, 670.3801879882812, 713.8804321289062, 727.5524291992188]]

# Open the image
with Image.open(images_dir) as img:
    draw = ImageDraw.Draw(img)

    # Draw red boxes for each set of coordinates
    for box in text_coords:
        # box format: [x1, y1, x2, y2]
        draw.rectangle(box, outline="red", width=5)

    # Save the modified image with a new name
    modified_image_path = os.path.join(test_save_path, f'modified_{os.path.basename(images_dir)}')
    img.save(modified_image_path)

print(f"Saved modified image: {modified_image_path}")

Saved modified image: ./modified_01.jpg
